In [3]:
!pip install numpy==1.25.0 gensim nltk optuna pyLDAvis pymorphy2 tqdm num2words wget corus

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 58.0 MB/s eta 0:00:00
  Using cached optuna-3.4.0-py3-none-any.whl (409 kB)
  Using cached pyLDAvis-3.4.1-py3-none-any.whl (2.6 MB)
  Using cached pymorphy2-0.9.1-py3-none-any.whl (55 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.3/143.3 kB 15.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.7/83.7 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.8/226.8 kB 17.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 80.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 90.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 341.8/341.8 kB 33.3 MB/s eta 0:00:00
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9655 sha256=0882df98282018abf7168f3e

In [4]:
import ast
import gc
import os
import random
import re
import string
import warnings
from functools import partial
from itertools import islice, product
from typing import Tuple

import nltk
import numpy as np
import optuna
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis
import wget
from corus import load_lenta
from gensim.corpora import Dictionary
from gensim.models import CoherenceModel, Phrases
from gensim.models.ldamodel import LdaModel
from nltk import WordPunctTokenizer
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize, word_tokenize
from num2words import num2words
from pymorphy2 import MorphAnalyzer
from tqdm import tqdm

warnings.filterwarnings("ignore")
pyLDAvis.enable_notebook()
nltk.download("punkt")
nltk.download("stopwords")

/usr/local/lib/python3.10/dist-packages/google/rpc/__init__.py:20: DeprecationWarning: Deprecated call to `pkg_resources.declare_namespace('google.rpc')`.
Implementing implicit namespace packages (as specified in PEP 420) is preferred to `pkg_resources.declare_namespace`. See https://setuptools.pypa.io/en/latest/references/keywords.html#keyword-namespace-packages
  pkg_resources.declare_namespace(__name__)
/usr/local/lib/python3.10/dist-packages/pkg_resources/__init__.py:2349: DeprecationWarning: Deprecated call to `pkg_resources.declare_namespace('google')`.
Implementing implicit namespace packages (as specified in PEP 420) is preferred to `pkg_resources.declare_namespace`. See https://setuptools.pypa.io/en/latest/references/keywords.html#keyword-namespace-packages
  declare_namespace(parent)
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping c

True

In [5]:
RANDOM_SEED = 42

In [6]:
random.seed(RANDOM_SEED)
os.environ["PYTHONHASHSEED"] = str(RANDOM_SEED)
np.random.seed(RANDOM_SEED)

In [7]:
morph_analyzer = MorphAnalyzer(lang="ru")
tokenizer = WordPunctTokenizer()

In [8]:
russian_stopwords = stopwords.words("russian")
additional_sw = "мои оно мной мною мог могут мор мое мочь оба нам нами ними однако нему никуда наш нею неё наша наше наши очень отсюда вон вами ваш ваша ваше ваши весь всем всеми вся ими ею будем будете будешь буду будь будут кому кого которой которого которая которые который которых кем каждое каждая каждые каждый кажется та те тому собой тобой собою тобою тою хотеть хочешь свое свои твой своей своего своих твоя твоё сама сами теми само самом самому самой самого самим самими самих саму чему тебе такое такие также такая сих тех ту эта это этому туда этим этими этих абы аж ан благо буде вроде дабы едва ежели затем зато ибо итак кабы коли коль либо лишь нежели пока покамест покуда поскольку притом причем пускай пусть ровно сиречь словно также точно хотя чисто якобы "
pronouns = "я мы ты вы он она оно они себя мой твой ваш наш свой его ее их то это тот этот такой таков столько весь всякий сам самый каждый любой иной другой кто что какой каков чей сколько никто ничто некого нечего никакой ничей нисколько кто-то кое-кто кто-нибудь кто-либо что-то кое-что что-нибудь что-либо какой-то какой-либо какой-нибудь некто нечто некоторый некий"
conjunctions = "что чтобы как когда ибо пока будто словно если потому что оттого что так как так что лишь только как будто с тех пор как в связи с тем что для того чтобы кто как когда который какой где куда откуда"
digits = "ноль один два три четыре пять шесть семь восемь девять десять одиннадцать двенадцать тринадцать четырнадцать пятнадцать шестнадцать семнадцать восемнадцать девятнадцать двадцать тридцать сорок пятьдесят шестьдесят семьдесят восемьдесят девяносто сто"
modal_words = "вероятно возможно видимо по-видимому кажется наверное безусловно верно  действительно конечно несомненно разумеется"
particles = "да так точно ну да не ни неужели ли разве а что ли что за то-то как ну и ведь даже еще ведь уже все все-таки просто прямо вон это вот как словно будто точно как будто вроде как бы именно как раз подлинно ровно лишь только хоть всего исключительно вряд ли едва ли"
prepositions = "близ  вблизи  вдоль  вокруг  впереди  внутрь  внутри  возле  около  поверх  сверху  сверх  позади  сзади  сквозь  среди  прежде  мимо  вслед  согласно  подобно  навстречу  против  напротив  вопреки  после  кроме  вместе  вдали  наряду  совместно  согласно  нежели вроде от бишь до без аж тех раньше совсем только итак например из прямо ли следствие а поскольку благо пускай благодаря случае затем притом также связи время при чтоб просто того невзирая даром вместо точно покуда тогда зато ради ан буде прежде насчет раз причине тому так даже исходя коль кабы более ровно либо помимо как-то будто если словно лишь бы и не будь пор тоже разве чуть как хотя наряду потому пусть в равно между сверх ибо на судя то чтобы относительно или счет за но сравнению причем оттого есть когда уж ввиду тем для дабы чем хоть с вплоть скоро едва после той да вопреки ежели кроме сиречь же коли под абы несмотря все пока покамест паче прямо-таки перед что по вдруг якобы подобно"
evaluative = "наиболее наименее лучший больший высший низший худший более менее"

russian_stopwords.extend(additional_sw.split())
russian_stopwords.extend(pronouns.split())
russian_stopwords.extend(conjunctions.split())
russian_stopwords.extend(digits.split())
russian_stopwords.extend(modal_words.split())
russian_stopwords.extend(particles.split())
russian_stopwords.extend(prepositions.split())
russian_stopwords.extend(evaluative.split())
russian_stopwords = set(russian_stopwords)

## Скачиваем датасет, делаем предобработку

In [9]:
dataset_url = "https://github.com/yutkin/Lenta.Ru-News-Dataset/releases/download/v1.0/lenta-ru-news.csv.gz"
wget.download(dataset_url)

'lenta-ru-news.csv.gz'

In [10]:
path = "lenta-ru-news.csv.gz"
records = load_lenta(path)

Датасет большой, поэтому возьмем 3000 записей

In [11]:
news = (record.title + ". " + record.text for record in islice(records, 3000))

### Делаем препроцессинг текста

In [12]:
def convert_digit_to_word(text):
    pattern = r"\d+"
    numbers = re.findall(pattern, text)
    for number in numbers:
        text = text.replace(number, num2words(number, lang="ru"))
    return text


def lemmatize(words) -> list[str]:
    return [morph_analyzer.parse(word)[0].normal_form for word in words]


def delete_stop_words(words):
    return [word for word in words if word not in russian_stopwords]


def delete_punctuation(text):
    return " ".join([word for word in tokenizer.tokenize(text) if word.isalnum()])


def tokenize_by_sentences(text: str) -> list[str]:
    return sent_tokenize(text)


def tokenize_by_words(text):
    return word_tokenize(text)


def convert_text_to_lowercase(text: str):
    return text.lower()


def preprocess(text: str) -> list[str]:
    sentences: list[str] = tokenize_by_sentences(text)
    preprocessed_sentences: list[str] = list()
    for sentence in sentences:
        sentence = convert_text_to_lowercase(sentence)
        sentence = convert_digit_to_word(sentence)
        sentence = delete_punctuation(sentence)
        words = tokenize_by_words(sentence)
        words = delete_stop_words(words)
        words = lemmatize(words)
        preprocessed_sentences.extend(words)
    return preprocessed_sentences

In [13]:
preprocessed_news = [preprocess(record) for record in tqdm(news)]

3000it [01:58, 25.39it/s]


## Обучение LDA

Подготовим словарь и корпус для LDA из датасета

In [14]:
bigram = Phrases(preprocessed_news, min_count=5, threshold=100)
bigram_documents = [bigram[doc] for doc in preprocessed_news]

dictionary = Dictionary(bigram_documents)
corpus = [dictionary.doc2bow(doc) for doc in bigram_documents]

Начинаем эксперименты по подбору параметров. Будем подбирать num_topics, chunk_size, passes в разных сочетаниях и проверять качество на coherence модели

In [15]:
def train(
    num_topics: int,
    alpha: float,
    eta: float,
    chunk_size: int,
    passes: int,
    dictionary: Dictionary,
    corpus,
):
    lda_model = LdaModel(
        corpus,
        num_topics=num_topics,
        id2word=dictionary,
        random_state=RANDOM_SEED,
        update_every=1,
        alpha=alpha,
        eta=eta,
        chunksize=chunk_size,
        passes=passes,
        iterations=150,
        per_word_topics=True,
    )

    perplexity = lda_model.log_perplexity(corpus)
    print(
        f"LDA с параметрами num_topics {num_topics}, alpha {alpha},  eta {eta}, chunksize {chunk_size} и passes {passes} -  perplexity: {perplexity}"
    )
    return perplexity, lda_model

In [16]:
def objective(trial, corpus, dictionary, texts):
    num_topics = trial.suggest_int("num_topics", 5, 25)
    alpha = trial.suggest_float("alpha", 0.01, 1.0)
    eta = trial.suggest_float("eta", 0.01, 1.0)
    passes = trial.suggest_int("passes", 1, 5)
    chunk_size = trial.suggest_int("chunk_size", 5, 30)
    coherence, _ = train(
        num_topics=num_topics,
        alpha=alpha,
        eta=eta,
        chunk_size=chunk_size,
        passes=passes,
        dictionary=dictionary,
        corpus=corpus,
    )
    return coherence

In [17]:
%%time
# https://groups.google.com/g/gensim/c/uQxQiR2oC98?pli=1
study = optuna.create_study(direction="maximize")
study.optimize(
    partial(objective, corpus=corpus, dictionary=dictionary, texts=preprocessed_news),
    n_trials=15,
    show_progress_bar=True,
)

best_trial = study.best_trial
best_params = study.best_params

[I 2023-11-02 15:44:06,244] A new study created in memory with name: no-name-fb24c890-13a7-4ceb-973a-1fa487524b84


  0%|          | 0/15 [00:00<?, ?it/s]

LDA с параметрами num_topics 21, alpha 0.3534491585492133,  eta 0.21942119012769604, chunksize 29 и passes 3 -  perplexity: -9.347177701915026
[I 2023-11-02 15:44:43,528] Trial 0 finished with value: -9.347177701915026 and parameters: {'num_topics': 21, 'alpha': 0.3534491585492133, 'eta': 0.21942119012769604, 'passes': 3, 'chunk_size': 29}. Best is trial 0 with value: -9.347177701915026.
LDA с параметрами num_topics 22, alpha 0.35307556384082583,  eta 0.27048541020179284, chunksize 13 и passes 2 -  perplexity: -9.36927636750939
[I 2023-11-02 15:45:32,795] Trial 1 finished with value: -9.36927636750939 and parameters: {'num_topics': 22, 'alpha': 0.35307556384082583, 'eta': 0.27048541020179284, 'passes': 2, 'chunk_size': 13}. Best is trial 0 with value: -9.347177701915026.
LDA с параметрами num_topics 24, alpha 0.8910107675168576,  eta 0.11243063229406546, chunksize 10 и passes 3 -  perplexity: -10.150802455980315
[I 2023-11-02 15:47:10,705] Trial 2 finished with value: -10.1508024559803

Обучим модель с лучшими параметрами

In [19]:
_, model = train(
    **best_params, dictionary=dictionary, corpus=corpus
)

LDA с параметрами num_topics 5, alpha 0.3579542317811885,  eta 0.5545560598078466, chunksize 25 и passes 5 -  perplexity: -8.998220318471995


In [20]:
def visualize_model(model: LdaModel, corpus: list[list[Tuple[int]]]):
    return gensimvis.prepare(model, corpus, dictionary=model.id2word, mds="mmds")

### Визуализируем самую лучшую модель

In [21]:
visualize_model(model=model, corpus=corpus)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1     -0.095899  0.186591       1        1  40.990723
0     -0.069856 -0.181475       2        1  26.265830
3     -0.199634 -0.004806       3        1  19.932940
4      0.144352 -0.119507       4        1   7.290555
2      0.221037  0.119197       5        1   5.519952, topic_info=            Term         Freq        Total Category  logprob  loglift
783      украина   688.000000   688.000000  Default  30.0000  30.0000
29        россия  2638.000000  2638.000000  Default  29.0000  29.0000
1465  украинский   556.000000   556.000000  Default  28.0000  28.0000
741      корабль   429.000000   429.000000  Default  27.0000  27.0000
22       процент   693.000000   693.000000  Default  26.0000  26.0000
...          ...          ...          ...      ...      ...      ...
699         сила   102.555410   179.316275   Topic5  -5.2830   2.3381
1078   задержать   147.455836   406.653004   Topic5  -4.9199   1.8824
1356     граница    95.742970   245.027417   Topic5  -5.3517   1.9571
146       ноябрь   175.283051  2155.113753   Topic5  -4.7470   0.3876
15       назвать    61.313775   567.476379   Topic5  -5.7974   0.6716

[312 rows x 6 columns], token_table=      Topic      Freq       Term
term                            
127       1  0.009159  instagram
127       2  0.979987  instagram
127       3  0.009159  instagram
127       4  0.009159  instagram
2453      1  0.022461        ufc
...     ...       ...        ...
7337      3  0.018530    ювентус
7337      4  0.963582    ювентус
9639      1  0.009859     ян_кап
9639      2  0.009859     ян_кап
9639      5  0.985873     ян_кап

[890 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 1, 4, 5, 3])

Посмотрим на топики модели

In [22]:
model.print_topics()

[(0,
  '0.007*"тысяча" + 0.006*"год" + 0.006*"один" + 0.005*"женщина" + 0.005*"человек" + 0.005*"мужчина" + 0.005*"сообщать" + 0.005*"ребёнок" + 0.005*"летний" + 0.004*"дом"'),
 (1,
  '0.017*"россия" + 0.010*"ноябрь" + 0.010*"российский" + 0.009*"тысяча" + 0.009*"год" + 0.008*"два" + 0.008*"страна" + 0.008*"заявить" + 0.007*"слово" + 0.006*"президент"'),
 (2,
  '0.034*"украина" + 0.027*"украинский" + 0.021*"корабль" + 0.012*"керченский_пролив" + 0.011*"фсб" + 0.011*"киев" + 0.011*"военный_положение" + 0.009*"порошенко" + 0.009*"ноябрь" + 0.007*"задержать"'),
 (3,
  '0.016*"тысяча" + 0.015*"год" + 0.011*"два" + 0.009*"процент" + 0.008*"компания" + 0.005*"рубль" + 0.005*"доллар" + 0.004*"новый" + 0.004*"миллион" + 0.003*"цена"'),
 (4,
  '0.010*"самолёт" + 0.006*"медведев" + 0.005*"матч" + 0.005*"команда" + 0.005*"два" + 0.004*"футболист" + 0.004*"победа" + 0.004*"спортсмен" + 0.004*"клуб" + 0.003*"поединок"')]

Вывод: возможно, нужно было подбирать другие параметры, потому что училась модель долго, но качество не очень